# NIDIS Visualization Tool

Kernel: `ILAB Kernel (PyTorch)`

Purpose: To visualize the indicator rasters produced by the NIDIS code base.

In [1]:
import sys
sys.path.append('../../nidis')

from nidis.model.Metadata import (
    DictofInitialToWord_Seasons,
    DictofNumNamePairs_Channels
)

import xarray as xr
import pathlib

import panel as pn
import panel.widgets as pnw
import hvplot.xarray
assert hvplot.xarray

pn.extension(loading_spinner='dots', template='bootstrap')
pn.param.ParamMethod.loading_indicator = True

In [2]:
nc_dir_str: str = '/panfs/ccds02/nobackup/projects/ilab/projects/NIDIS/Outputs_fame'

nc_dir: pathlib.Path = pathlib.Path(nc_dir_str)

ncs_in_dir: list = sorted(nc_dir.glob('*.nc'))

indicator_dict: dict = {}

for nc_file_path in ncs_in_dir:
    subsplit = nc_file_path.stem.split('_')
    key = '_'.join(subsplit[6:-1])
    season = DictofInitialToWord_Seasons[subsplit[5]]
    if not key in indicator_dict:
        indicator_dict[key] = {}
    indicator_dict[key][season] = [str(nc_file_path)]

In [3]:
indicator_select = pnw.Select(
    name='Indicator',
    options=list(indicator_dict.keys())
)

season_select = pnw.Select(
    name='Season',
    options=list(indicator_dict[indicator_select.value].keys())
)

def update_callback(event):
    selected_first = event.new
    previous_value = season_select.value
    season_select.options = list(indicator_dict[selected_first].keys())
    season_select.value = previous_value if previous_value in season_select.options else season_select.options[0]


indicator_select.param.watch(update_callback, 'value')

Watcher(inst=Select(name='Indicator', options=['prcp_02_nCG', ...], value='prcp_02_nCG'), cls=<class 'panel.widgets.select.Select'>, fn=<function update_callback at 0x146621130cc0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [7]:
visible = pnw.Select(options=[True, False],
                     value=True,
                     name='Raster Visible')

"""

# selector = pnw.NestedSelect(options=indicator_dict,
#                  levels=['Indicator', 'Season', 'File'],
#                  layout=pn.Row)
def display_selection(selector_values: dict):
    indicator = selector_values['Indicator']
    season = selector_values['Season']
    file_path = selector_values['File']
    sampleArray = xr.load_dataset(file_path)
    rasterLayer = sampleArray['FracI'].hvplot(
                geo=True,
                cmap='jet',
                rasterize=True,
                frame_width=1200,
                tiles='EsriImagery',
                visible=visible)
    yield rasterLayer
"""
    
def display_selection(indicator_selection: str, season_selection: str):
    print(indicator_selection)
    print(season_selection)
    file_path = indicator_dict[indicator_selection][season_selection]
    print(file_path)
    sampleArray = xr.load_dataset(file_path[0])
    rasterLayer = sampleArray['FracI'].hvplot(
                geo=True,
                cmap='jet',
                rasterize=True,
                frame_width=1200,
                tiles='EsriImagery',
                visible=visible)
    yield rasterLayer
    

In [8]:
pn.Column(pn.Row(visible, indicator_select, season_select), pn.bind(display_selection, indicator_select, season_select)).servable()

Column
    [0] Row
        [0] Select(name='Raster Visible', options=[True, False], value=True)
        [1] Select(name='Indicator', options=['prcp_02_nCG', ...], value='prcp_02_nCG')
        [2] Select(name='Season', options=['All-season', ...], value='All-season')
    [1] ParamFunction(function, _pane=Str, defer_load=False, loading_indicator=True)

prcp_02_nCG
All-season
['/panfs/ccds02/nobackup/projects/ilab/projects/NIDIS/Outputs_fame/NC2D_From_nCG1D_FIs1X1Etc_113_A_prcp_02_nCG_V2.nc']


In [6]:
#!pip install netCDF4 h5netcdf